In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.proxy import Proxy, ProxyType
import requests
from dotenv import load_dotenv
from os import environ
import random
import pandas as pd
import time
from fake_useragent import UserAgent

In [ ]:
load_dotenv()
random.seed()

In [ ]:
path = environ["WEBDRIVER_PATH"]
url = environ["BASE_URL"]
feed_endpoint = environ["FEED_ENDPOINT"]
login_uid_xpath = '//*[@id="session_key"]' 
login_password_xpath = '//*[@id="session_password"]'
login_submit_xpath = '//*[@id="main-content"]/section[1]/div/div/form/button'

In [ ]:
phone_skip_btn = "/html/body/div/div[1]/section/div[2]/div/article/footer/div/div/button"

In [ ]:
# options = Options()
# ua = UserAgent()
# user_agent = ua.random

# options.add_argument(f'user-agent={user_agent}')
driver = webdriver.Chrome(
        # options=options, 
        executable_path=path
    )
driver.get(url)
uid_field = driver.find_element(By.XPATH, login_uid_xpath)
uid_field.send_keys(environ["LINKEDIN_USERNAME"])
password_field = driver.find_element(By.XPATH, login_password_xpath)
password_field.send_keys(environ["LINKEDIN_PASSWORD"])
okay_btn = driver.find_element(By.XPATH, login_submit_xpath).click()
if environ["ADD_PHONE_ENDPOINT"] in driver.current_url:
    btn = driver.find_element(By.XPATH, phone_skip_btn)
    btn.click()
    btn = None
time.sleep(5)
driver.quit()

In [ ]:
with open("../data_store/uids.csv", "rt", encoding="utf-8") as data:
    uid_list = data.read().split("\n")
uid_list.sort()
print(uid_list)

## Traversing JSON feed data for own profile

We will now, parse the json file and take only the necessary details.

In [34]:
import json

file_path = "/home/prithoo/Coding/B2B_ProofOfConcept/output_data/search_results/prithoo11335@gmail.com_feed.json"

with open(file_path)as json_file:
    feed = json.load(json_file)

feed = feed["feed"]

feed_list = []
for post in feed:
    if "position" in post.get("content"):
        data_dict = {}
        data_dict["author"] = post.get("author_profile", "")
        data_dict["post"] = post.get("content", "")
        data_dict["urn"] = post.get("url", "").split("/")[-1]
        data_dict["link"] = post.get("url", "").lower()
        feed_list.append(data_dict)

# print(feed_list)

In [36]:
import pandas as pd

df = pd.DataFrame.from_dict(feed_list)
df.head()
op_file_name = f"/home/prithoo/Coding/B2B_ProofOfConcept/cleaned_data/{file_path.split('/')[-1]}.csv"
df.to_csv(op_file_name)